In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
# READ AND DROP COLUMNS
df = pd.read_csv("D:/DM_Project_Dataset/training_cleaned.csv")
df = df.loc[:, ['Nationality', 'TopThreeAmericanName', 'Make', 'RetailAVG', 'AuctionAVG', 'IsBadBuy']]
df.head()

Nationality TopThreeAmericanName      Make  RetailAVG  AuctionAVG  IsBadBuy
0  OTHER ASIAN                OTHER       KIA    10300.0      7128.5         1
1     AMERICAN             CHRYSLER     DODGE     6948.0      5970.5         0
2     AMERICAN             CHRYSLER     DODGE    11199.0      8061.5         0
3     AMERICAN                 FORD      FORD     6696.5      5737.5         0
4     AMERICAN             CHRYSLER  CHRYSLER     7046.0      6061.0         0

In [3]:
# y = df['IsBadBuy']
# X = df_clas
y = df['IsBadBuy']
X = df.drop('IsBadBuy',axis=1)

In [4]:
# define categorical columns
cat_cols = ['Nationality', 'TopThreeAmericanName', 'Make']

# one-hot encode categorical columns
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), cat_cols)], remainder='passthrough')
X = ct.fit_transform(X)

In [5]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.5, 1]
}

scores = ['recall', 'f1']

for score in scores:
    print("# Tuning hyper-parameters for ----> %s" % score)
    print()
    
    obj = xgb.XGBClassifier()
    
    if (score == "recall"):
      clf = GridSearchCV(obj, tuned_parameters, cv=5, scoring=score)
    else:
      clf = GridSearchCV(obj, tuned_parameters, cv=5, scoring='%s_macro' % score)
                         
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for ----> recall



KeyboardInterrupt: 